In [1]:
import sys
#!{sys.executable} -m pip install pandas numpy scikit-learn lightgbm matplotlib duckdb 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import warnings
import duckdb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (confusion_matrix, auc, roc_curve, accuracy_score, 
                             precision_score, recall_score, f1_score, 
                             precision_recall_curve, roc_auc_score, brier_score_loss)
from lightgbm import LGBMClassifier
import lightgbm as lgb
#pd.set_option('display.max_rows', None)
random.seed(37)
np.random.seed(37)
warnings.filterwarnings("ignore")

con = duckdb.connect(database=":memory:")

print("Setup complete")

c:\Users\vchaudha\AppData\Local\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


Setup complete


### control panel

In [2]:
tables_location="C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main"

race_map = {
    'White': 'White',
    'Black or African American': 'Black',
    'Asian': 'Asian',
    'Other': 'Others',
    'Unknown': 'Others',
    'Did Not Encounter': 'Others',
    'Refusal': 'Others',
    'American Indian or Alaska Native': 'Others',
    'Native Hawaiian or Other Pacific Islander': 'Others',
    np.nan: 'Others'
}

ethnicity_map = {
    'Not Hispanic or Latino': 'Not Hispanic or Latino',
    'Hispanic or Latino': 'Hispanic or Latino',
    'Did Not Encounter': 'Others',
    'Refusal': 'Others',
    '*Unspecified': 'Others',
    np.nan: 'Others'
}

site_name='RUSH'

finetune=True

### ICU close to Admission

1. Check ICU location_category between admission_dttmtime and 48hr stop from admission
2. Check ICU stay at least 24 hr (for ICU - OR - ICU including OR in ICU stay 24hr)

In [3]:
location = pd.read_csv(f"{tables_location}/rclif/clif_adt.csv")
encounter = pd.read_csv(f"{tables_location}/rclif/clif_encounter_demographics_dispo.csv")
limited=pd.read_csv(f"{tables_location}/rclif/clif_limited_identifiers.csv")
demog=pd.read_csv(f"{tables_location}/rclif/clif_patient_demographics.csv")

In [4]:
join=pd.merge(location[['encounter_id','location_category','in_dttm','out_dttm']],\
              limited[['encounter_id','admission_dttm']], on=['encounter_id'], how='left')

icu_data=pd.merge(join,\
                  encounter[['encounter_id','age_at_admission','disposition']], on=['encounter_id'], how='left')


icu_data['in_dttm'] = pd.to_datetime(icu_data['in_dttm'])
icu_data['admission_dttm'] = pd.to_datetime(icu_data['admission_dttm'])
icu_data['out_dttm'] = pd.to_datetime(icu_data['out_dttm'])

icu_48hr_check = icu_data[
    (icu_data['location_category'] == 'ICU') &
    (icu_data['in_dttm'] >= icu_data['admission_dttm']) &
    (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=48)) &
    (icu_data['in_dttm'].dt.year >= 2020) & (icu_data['in_dttm'].dt.year <= 2022)
]['encounter_id'].unique()

icu_data=icu_data[icu_data['encounter_id'].isin(icu_48hr_check) & (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=72))].reset_index(drop=True)

icu_data = icu_data.sort_values(by=['in_dttm']).reset_index(drop=True)

icu_data["RANK"]=icu_data.sort_values(by=['in_dttm'], ascending=True).groupby("encounter_id")["in_dttm"].rank(method="first", ascending=True).astype(int)


min_icu=icu_data[icu_data['location_category'] == 'ICU'].groupby('encounter_id')['RANK'].min()
icu_data=pd.merge(icu_data, pd.DataFrame(zip(min_icu.index, min_icu.values), columns=['encounter_id', 'min_icu']), on='encounter_id', how='left')
icu_data=icu_data[icu_data['RANK']>=icu_data['min_icu']].reset_index(drop=True)

icu_data.loc[icu_data['location_category'] == 'OR', 'location_category'] = 'ICU'

icu_data['group_id'] = (icu_data.groupby('encounter_id')['location_category'].shift() != icu_data['location_category']).astype(int)
icu_data['group_id'] = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby('encounter_id')['group_id'].cumsum()


icu_data = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby(['encounter_id', 'location_category', 'group_id']).agg(
    min_in_dttm=('in_dttm', 'min'),
    max_out_dttm=('out_dttm', 'max'),
    admission_dttm=('admission_dttm', 'first'),
    age=('age_at_admission', 'first'),
    dispo=('disposition', 'first')
).reset_index()

min_icu=icu_data[icu_data['location_category'] == 'ICU'].groupby('encounter_id')['group_id'].min()
icu_data=pd.merge(icu_data, pd.DataFrame(zip(min_icu.index, min_icu.values), columns=['encounter_id', 'min_icu']), on='encounter_id', how='left')

icu_data=icu_data[(icu_data['min_icu']==icu_data['group_id']) &
         (icu_data['max_out_dttm']-icu_data['min_in_dttm'] >= pd.Timedelta(hours=24))
         ].reset_index(drop=True)


icu_data['after_24hr']=icu_data['min_in_dttm'] + pd.Timedelta(hours=24)

icu_data=icu_data[['encounter_id','min_in_dttm','after_24hr','age','dispo']]

icu_data=pd.merge(icu_data,\
                  demog, on=['encounter_id'], how='left')[['encounter_id','min_in_dttm','after_24hr','age','dispo','sex','ethnicity','race']]
icu_data=icu_data[~icu_data['sex'].isna()].reset_index(drop=True)
icu_data['isfemale']=(icu_data['sex'].str.lower() == 'female').astype(int)
icu_data['isdeathdispo'] = (icu_data['dispo'].str.contains('dead|expired', case=False, regex=True)).astype(int)

icu_data['ethnicity'] = icu_data['ethnicity'].map(ethnicity_map)
icu_data['race'] = icu_data['race'].map(race_map)


del location,encounter,limited,demog

### Vitals

In [5]:
vitals = con.execute(f'''
    SELECT 
        encounter_id,
        CAST(recorded_dttm AS datetime) AS recorded_time,
        CAST(vital_value AS float) AS vital_value,
        vital_name 
    FROM 
        read_csv_auto('{tables_location}/rclif/clif_vitals.csv')
    WHERE 
        vital_name IN ('weight_kg', 'pulse', 'sbp', 'dbp', 'temp_c','height_inches') 
        AND encounter_id IN (SELECT DISTINCT encounter_id FROM icu_data);
''').df()

vitals=con.execute('''
PIVOT vitals
ON vital_name
USING first(vital_value)
GROUP BY encounter_id,recorded_time;
''').df()

vitals['height_meters'] = vitals['height_inches'] * 0.0254

# Calculate BMI
vitals['bmi'] = vitals['weight_kg'] / (vitals['height_meters'] ** 2)


icu_data_agg=pd.merge(icu_data,vitals, on=['encounter_id'], how='left')
icu_data_agg=icu_data_agg[(icu_data_agg['recorded_time'] >= icu_data_agg['min_in_dttm']) & (icu_data_agg['recorded_time'] <= icu_data_agg['after_24hr'])].reset_index(drop=True)

icu_data_agg = icu_data_agg.groupby(['encounter_id']).agg(
    min_bmi=('bmi', 'min'),
    max_bmi=('bmi', 'max'),
    avg_bmi=('bmi', 'mean'),
    min_weight_kg=('weight_kg', 'min'),
    max_weight_kg=('weight_kg', 'max'),
    avg_weight_kg=('weight_kg', 'mean'),
    min_pulse=('pulse', 'min'),
    max_pulse=('pulse', 'max'),
    avg_pulse=('pulse', 'mean'),
    min_sbp=('sbp', 'min'),
    max_sbp=('sbp', 'max'),
    avg_sbp=('sbp', 'mean'),
    min_dbp=('dbp', 'min'),
    max_dbp=('dbp', 'max'),
    avg_dbp=('dbp', 'mean'),
    min_temp_c=('temp_c', 'min'),
    max_temp_c=('temp_c', 'max'),
    avg_temp_c=('temp_c', 'mean'),
).reset_index()

icu_data=pd.merge(icu_data,icu_data_agg, on=['encounter_id'], how='left')

del vitals,icu_data_agg

### Labs

In [6]:
labs = con.execute(f'''
    SELECT 
        encounter_id,
        CAST(lab_order_dttm AS datetime) AS lab_order_dttm,
        TRY_CAST(lab_value AS float) AS lab_value,
        lab_name 
    FROM 
        read_csv_auto('{tables_location}/rclif/clif_labs.csv')
    WHERE 
        ((lab_name='monocyte' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_name='lymphocyte' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_name='basophil' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_name='neutrophil' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_name='albumin' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_name='ast' and reference_unit = 'U/L' and lab_type_name='standard') OR
        (lab_name='total_protein' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_name='alkaline_phosphatase' and reference_unit = 'U/L' and lab_type_name='standard') OR
        (lab_name='bilirubin_total' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_name='bilirubin_conjugated' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_name='calcium' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_name='chloride' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_name='potassium' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_name='sodium' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_name='glucose_serum' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_name='hemoglobin' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_name='platelet count' and reference_unit = 'K/uL' and lab_type_name='standard') OR
        (lab_name='wbc' and reference_unit = 'K/uL' and lab_type_name='standard'))  
        AND encounter_id IN (SELECT DISTINCT encounter_id FROM icu_data);
''').df()

labs=con.execute('''
PIVOT labs
ON lab_name
USING first(lab_value)
GROUP BY encounter_id,lab_order_dttm;
''').df()

In [7]:

icu_data_agg=pd.merge(icu_data,labs, on=['encounter_id'], how='left')
icu_data_agg=icu_data_agg[(icu_data_agg['lab_order_dttm'] >= icu_data_agg['min_in_dttm']) & (icu_data_agg['lab_order_dttm'] <= icu_data_agg['after_24hr'])].reset_index(drop=True)

Lab_variables = [
   'albumin', 'alkaline_phosphatase',
       'ast', 'basophil', 'bilirubin_conjugated', 'bilirubin_total', 'calcium',
       'chloride', 'glucose_serum', 'hemoglobin', 'lymphocyte', 'monocyte',
       'neutrophil', 'platelet count', 'potassium', 'sodium', 'total_protein',
       'wbc'
]
agg_dict = {var: ['min', 'max', 'mean'] for var in Lab_variables}

icu_data_agg = icu_data_agg.groupby('encounter_id').agg(agg_dict).reset_index()

icu_data_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in icu_data_agg.columns.values]



In [8]:
icu_data=pd.merge(icu_data,icu_data_agg, on=['encounter_id'], how='left')

#### model

In [9]:
model_col=['isfemale','age', 'min_bmi', 'max_bmi', 'avg_bmi',
       'min_weight_kg', 'max_weight_kg', 'avg_weight_kg', 'min_pulse',
       'max_pulse', 'avg_pulse', 'min_sbp', 'max_sbp', 'avg_sbp', 'min_dbp',
       'max_dbp', 'avg_dbp', 'min_temp_c', 'max_temp_c', 'avg_temp_c',
       'albumin_min', 'albumin_max', 'albumin_mean',
       'alkaline_phosphatase_min', 'alkaline_phosphatase_max',
       'alkaline_phosphatase_mean', 'ast_min', 'ast_max', 'ast_mean',
       'basophil_min', 'basophil_max', 'basophil_mean',
       'bilirubin_conjugated_min', 'bilirubin_conjugated_max',
       'bilirubin_conjugated_mean', 'bilirubin_total_min',
       'bilirubin_total_max', 'bilirubin_total_mean', 'calcium_min',
       'calcium_max', 'calcium_mean', 'chloride_min', 'chloride_max',
       'chloride_mean', 'glucose_serum_min', 'glucose_serum_max',
       'glucose_serum_mean', 'hemoglobin_min', 'hemoglobin_max',
       'hemoglobin_mean', 'lymphocyte_min', 'lymphocyte_max',
       'lymphocyte_mean', 'monocyte_min', 'monocyte_max', 'monocyte_mean',
       'neutrophil_min', 'neutrophil_max', 'neutrophil_mean',
       'platelet count_min', 'platelet count_max', 'platelet count_mean',
       'potassium_min', 'potassium_max', 'potassium_mean', 'sodium_min',
       'sodium_max', 'sodium_mean', 'total_protein_min', 'total_protein_max',
       'total_protein_mean', 'wbc_min', 'wbc_max', 'wbc_mean']


model=lgb.Booster(model_file=f'{tables_location}/projects/Mortality_model/models/lgbm_model_20240425-112249.txt')

#### basic metrics

In [10]:
X_test=icu_data[model_col]
y_test=icu_data['isdeathdispo']

y_pred_proba = model.predict(X_test)
icu_data['pred_proba'] = y_pred_proba
# Calculate metrics at default threshold (0.5)

accuracy = accuracy_score(y_test, (y_pred_proba >= 0.5).astype(int))
recall = recall_score(y_test, (y_pred_proba >= 0.5).astype(int))
precision = precision_score(y_test, (y_pred_proba >= 0.5).astype(int))
roc_auc = roc_auc_score(y_test, y_pred_proba)
brier_score = brier_score_loss(y_test, y_pred_proba)


results_Metric = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'Precision', 'ROC AUC', 'Brier Score Loss'],
    'Value': [accuracy, recall, precision, roc_auc, brier_score],
    'SiteName': [f'{site_name}'] * 5
})

results_Metric.to_csv(f'output/result_metrics_{site_name}.csv',index=False)
results_Metric

,Metric,Value,SiteName
0,Accuracy,0.957052,RUSH
1,Recall,0.457457,RUSH
2,Precision,0.936330,RUSH
3,ROC AUC,0.951201,RUSH
4,Brier Score Loss,0.032766,RUSH


#### probablity table

In [11]:
prob_df_lgbm = pd.DataFrame({'site_label ':y_test, 'site_proba': y_pred_proba,'Site_name':f"{site_name}" })
prob_df_lgbm.to_csv(f'output/Model_probabilities_{site_name}.csv',index=False)
prob_df_lgbm.head()

,site_label,site_proba,Site_name
0,0,0.022355,RUSH
1,0,0.012466,RUSH
2,0,0.023490,RUSH
3,0,0.005411,RUSH
4,0,0.025744,RUSH


#### model fairness test accross 'race', 'ethnicity', 'sex'

In [12]:
def calculate_metrics(data, true_col, pred_prob_col, subgroup_cols):
    results = []
    total_count = len(data)

    for subgroup_col in subgroup_cols:
       
        filtered_data = data.dropna(subset=[subgroup_col])
        
        for group in filtered_data[subgroup_col].unique():
            subgroup_data = filtered_data[filtered_data[subgroup_col] == group]
            group_count = len(subgroup_data)
            proportion = group_count / total_count

            if np.unique(subgroup_data[true_col]).size > 1:  # Check if both classes are present
                auc = roc_auc_score(subgroup_data[true_col], subgroup_data[pred_prob_col])
                tn, fp, fn, tp = confusion_matrix(subgroup_data[true_col], (subgroup_data[pred_prob_col] > 0.5).astype(int)).ravel()
                ppv = tp / (tp + fp) if (tp + fp) != 0 else 0
                result = {'Subgroup': subgroup_col, 'Group': group, 'AUC': auc, 'PPV': ppv, 'Group Count': group_count, 'Total Count': total_count, 'Proportion': proportion, 'site_name':f'{site_name}'}
            else:
                result = {'Subgroup': subgroup_col, 'Group': group, 'AUC': 'Not defined', 'PPV': 'Not applicable', 'Group Count': group_count, 'Total Count': total_count, 'Proportion': proportion, 'site_name':f'{site_name}'}
            
            results.append(result)
    
   
    results_df = pd.DataFrame(results)
    return results_df

result_df = calculate_metrics(icu_data, 'isdeathdispo', 'pred_proba', ['race', 'ethnicity', 'sex'])

In [13]:
result_df.to_csv(f'output/fairness_test_{site_name}.csv',index=False)
result_df

,Subgroup,Group,AUC,PPV,Group Count,Total Count,Proportion,site_name
0,race,White,0.950923,0.942708,5621,14599,0.385026,RUSH
1,race,Others,0.953310,0.932773,2731,14599,0.187068,RUSH
2,race,Black,0.948128,0.930000,5762,14599,0.394685,RUSH
3,race,Asian,0.964165,0.956522,485,14599,0.033221,RUSH
4,ethnicity,Not Hispanic or Latino,0.952603,0.937965,11621,14599,0.796013,RUSH
5,ethnicity,Hispanic or Latino,0.946807,0.926829,2874,14599,0.196863,RUSH
6,ethnicity,Others,0.899901,1.000000,104,14599,0.007124,RUSH
7,sex,Male,0.950824,0.932384,7779,14599,0.532845,RUSH
8,sex,Female,0.951510,0.940711,6820,14599,0.467155,RUSH


#### thrshold check at site

In [14]:
def top_n_percentile(target_var, pred_proba):
    #thr_list = [0.99,0.97, 0.95,0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10]
    thr_list = np.arange(1, 0, -0.01)
    col = ['N Percentile', 'Thr Value','TN','FP','FN','TP','Sensitivity','Specificity','PPV', 'NPV' ,'Recall','Accuracy','site_name']
    result = pd.DataFrame(columns = col)
    i = 0
    
    for thr in thr_list: 
        prob = pd.DataFrame()
        prob['target_var'] = target_var
        prob['pred_proba'] = pred_proba

        thr_value = prob['pred_proba'].quantile(thr)
        prob['pred_proba_bin'] = np.where(prob['pred_proba'] >= thr_value, 1, 0)
        tn,fp,fn,tp = confusion_matrix(prob['target_var'], prob['pred_proba_bin']).ravel()

        sensitivity = tp/(tp+fn)
        specificity = tn/(tn+fp)
        ppv = tp/(tp+fp)
        npv = tn/(tn+fn)
        recall = tp/(tp+fn)
        acc = (tp+tn)/(tp+fn+tn+fp)
        n_prec = 'Top '+ str(np.round((1 - thr) * 100,0))+ "%"
        result.loc[i] = [n_prec,thr_value,tn,fp,fn,tp,sensitivity,specificity ,ppv,npv, recall, acc,f'{site_name}']
        i+=1
    return result
topn=top_n_percentile(y_test,y_pred_proba)

In [15]:
topn.to_csv(f'output/Top_N_percentile_PPV_{site_name}.csv',index=False)
topn.head(6)

,N Percentile,Thr Value,TN,FP,FN,TP,Sensitivity,Specificity,PPV,NPV,Recall,Accuracy,site_name
0,Top 0.0%,0.989530,13506,0,1092,1,0.000915,1.000000,1.000000,0.925195,0.000915,0.925200,RUSH
1,Top 1.0%,0.777559,13504,2,949,144,0.131747,0.999852,0.986301,0.934339,0.131747,0.934859,RUSH
2,Top 2.0%,0.657780,13494,12,813,280,0.256176,0.999112,0.958904,0.943175,0.256176,0.943489,RUSH
3,Top 3.0%,0.556040,13485,21,676,417,0.381519,0.998445,0.952055,0.952263,0.381519,0.952257,RUSH
4,Top 4.0%,0.464467,13465,41,550,543,0.496798,0.996964,0.929795,0.960756,0.496798,0.959518,RUSH
5,Top 5.0%,0.378441,13429,77,440,653,0.597438,0.994299,0.894521,0.968275,0.597438,0.964587,RUSH


FineTune

In [16]:
if finetune:
    train_data, test_data = train_test_split(icu_data, test_size=0.5, random_state=42)
    X_train=train_data[model_col]
    y_train=train_data['isdeathdispo']

    #test
    X_test=test_data[model_col]
    y_test=test_data['isdeathdispo']

    lgb_train = lgb.Dataset(X_train, y_train)

    params = {
        "boosting_type": "gbdt",
        "objective": "binary",
        "metric": "binary_logloss",
        "num_leaves": 31,
        "learning_rate": 0.05,
        "max_depth":-1}
    gbm = lgb.train(params, lgb_train, num_boost_round=10, init_model=f'{tables_location}/projects/Mortality_model/models/lgbm_model_20240425-112249.txt')

    y_pred_proba_ft = gbm.predict(X_test)

    accuracy = accuracy_score(y_test, (y_pred_proba_ft >= 0.5).astype(int))
    recall = recall_score(y_test, (y_pred_proba_ft >= 0.5).astype(int))
    precision = precision_score(y_test, (y_pred_proba_ft >= 0.5).astype(int))
    roc_auc = roc_auc_score(y_test, y_pred_proba_ft)
    brier_score = brier_score_loss(y_test, y_pred_proba_ft)


    results_Metric = pd.DataFrame({
        'Metric': ['Accuracy', 'Recall', 'Precision', 'ROC AUC', 'Brier Score Loss'],
        'Value': [accuracy, recall, precision, roc_auc, brier_score],
        'SiteName': [f'{site_name}'] * 5,
        'FineTune': ['Yes'] * 5,
    })
    results_Metric.to_csv(f'output/result_metrics_{site_name}_ft.csv',index=False)


    model_filename = f"output/lgbm_model_{site_name}_ft.txt"

    # Save the model using LightGBM's built-in function
    model.save_model(model_filename)

    print(results_Metric)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 555, number of negative: 6744
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004064 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12983
[LightGBM] [Info] Number of data points in the train set: 7299, number of used features: 74
             Metric     Value SiteName FineTune
0          Accuracy  0.956438     RUSH      Yes
1            Recall  0.457249     RUSH      Yes
2         Precision  0.904412     RUSH      Yes
3           ROC AUC  0.953286     RUSH      Yes
4  Brier Score Loss  0.032374     RUSH      Yes
